In [1]:
# dataset https://www.coursera.org/learn/ml-foundations/supplement/6DeQc/retrieving-wikipedia-articles-assignment

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("./input/people_wiki.csv")

In [4]:
df.head(2)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...


# 1. Top word count words for Elton John

In [5]:
elton=df[df["name"]=="Elton John"]

In [6]:
# count vecotirzer :D 
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

from sklearn.feature_extraction.text import CountVectorizer

In [7]:
count_vectorizer = CountVectorizer()
count = count_vectorizer.fit_transform(df['text'])

In [8]:
analyzer = count_vectorizer.build_analyzer()

In [10]:
vocabulary = count_vectorizer.vocabulary_
reverse_vocabulary = dict(zip(vocabulary.values(),vocabulary.keys()))

In [13]:
def count_words(doc):
    dic = {}
    if analyzer(doc):
        M = count_vectorizer.fit_transform([doc]).toarray()[0]
        for word,index in count_vectorizer.vocabulary_.items(): #writing here vocabulary instead of count_vectorizer.vocabulary_ it doesn't work ¿?
            dic[word] = M[index]
    return dic

In [14]:
elton['word_count'] = elton['text'].apply(count_words)

/home/vargas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
elton

,URI,name,text,word_count
19923,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...,"{'sir': 1, 'elton': 3, 'hercules': 1, 'john': ..."


In [16]:
# creating a table with dict of words and occurrences
elton_count_table = pd.DataFrame(elton['word_count'].values[0].items(),columns=['word','count'])
elton_count_table.sort_values(by='count',ascending=False).head()

,word,count
56,the,27
60,in,18
21,and,15
55,of,13
25,has,9


# 2. Top TF-IDF words for Elton John

In [ ]:
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf
# term frequency–inverse document frequency, the tf–idf value increases proportionally to the number of 
# times a word appears in the document and is offset by the number of documents in the corpus that 
# contain the word, which helps to adjust for the fact that some words appear more frequently in general.

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
tfidftransformer = TfidfTransformer(norm=None)
tfidf_weight = tfidftransformer.fit_transform(count)

In [24]:
def compute_tfidf(count,tfidf_M,voc):
    ans = []
    num = count.shape[0]
    for i in range(0,num):
        idx = np.where(count[i]!=np.zeros(count[i].shape))[1]
        dic = {}
        dic = dict((voc[index],tfidf_M[i,index]) for index in idx)
        ans.append(dic)
    return ans

In [27]:
word = [reverse_vocabulary[i] for i in tfidf_weight[elton.index[0]].indices]

In [28]:
elton_count_table = pd.DataFrame({'word':word,'tfidf':tfidf_weight[elton.index[0]].data})
elton_count_table.sort_values(by='tfidf',ascending =False)

,word,tfidf
24,the,27.002743
192,billboard,21.298630
105,john,20.938563
161,elton,20.465179
140,furnish,20.081204
...,...,...
9,who,1.909870
10,which,1.767411
148,first,1.695588
142,from,1.293750


In [ ]:
# ¿?

# 3. The cosine distance between 'Elton John's and 'Victoria Beckham's articles (represented with TF-IDF) falls within which range?

In [29]:
from sklearn.metrics.pairwise import cosine_distances

In [31]:
victoria = df[df['name'] == 'Victoria Beckham']

In [47]:
# in previous quizz is calculated "mathematically" the cosine distance from users
cosine_distances(tfidf_weight[elton.index[0]],tfidf_weight[victoria.index[0]]) # this is selecting the whole row

array([[0.85192118]])

# 4. The cosine distance between 'Elton John's and 'Paul McCartney's articles (represented with TF-IDF) falls within which range?

In [34]:
paul = df[df['name'] == 'Paul McCartney']

In [35]:
cosine_distances(tfidf_weight[elton.index[0]],tfidf_weight[paul.index[0]])

array([[0.69231325]])

# 5. Who is closer to 'Elton John', 'Victoria Beckham' or 'Paul McCartney'?

# 6. Who is the nearest cosine-distance neighbor to 'Elton John' using raw word counts?

In [48]:
from sklearn.neighbors import KNeighborsClassifier

In [53]:
knn_model_wordcounts = KNeighborsClassifier(algorithm='brute',metric='cosine')
knn_model_wordcounts.fit(count,df['name'])

KNeighborsClassifier(algorithm='brute', metric='cosine')

In [56]:
dist,ind = knn_model_wordcounts.kneighbors(tfidf_weight[elton.index[0]])
#pd.DataFrame({'reference_label':df.ix[ind[0]]['name'],'distance':dist[0]})

In [52]:
knn_model_tfidf = KNeighborsClassifier(algorithm='brute',metric='cosine')
knn_model_tfidf.fit(tfidf_weight,df['name'])

KNeighborsClassifier(algorithm='brute', metric='cosine')

KNeighborsClassifier(algorithm='brute', metric='cosine')

# Who is the nearest cosine-distance neighbor to 'Elton John' using TF-IDF?

# Who is the nearest cosine-distance neighbor to 'Victoria Beckham' using raw word counts?

# Who is the nearest cosine-distance neighbor to 'Victoria Beckham' using TF-IDF?